# Check out specific fungal taxa in the context of some metadata

In [3]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries

In [4]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries


In [5]:
from qiime2 import Visualization
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.feature_table.methods import (filter_samples, filter_seqs)
from sklearn.metrics import r2_score
import numpy as np
import re

from qiime2.plugins import diversity as q2d

%matplotlib inline

# Load data

--> Fungi OTU clustered and already annotated! 

In [6]:
df_ITS = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/berries-closedRefOTU/berries_harvest_cOTUs_rarefied_labled.tsv', sep='\t', index_col=0)
df_ITS.index.name = "SAMPLE_NAME"

df_ITS

,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Schizophyllaceae;g__Schizophyllum;s__Schizophyllum_commune,k__Fungi;p__Ascomycota;c__unidentified;o__unidentified;f__unidentified;g__unidentified;s__unidentified,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Crepidotaceae;g__Crepidotus;s__Crepidotus_stenocystis,k__Fungi;p__Basidiomycota;c__Tremellomycetes;o__Filobasidiales;f__Filobasidiaceae;g__Filobasidium;s__Filobasidium_magnum,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Sporidiobolales;f__Sporidiobolaceae;g__Sporobolomyces;s__Sporobolomyces_roseus,k__Fungi;p__Basidiomycota;c__Microbotryomycetes;o__Microbotryomycetes_ord_Incertae_sedis;f__Microbotryomycetes_fam_Incertae_sedis;g__Curvibasidium;s__Curvibasidium_cygneicollum,k__Fungi;p__Basidiomycota;c__Tremellomycetes;o__Tremellales;f__Bulleribasidiaceae;g__Derxomyces;s__Derxomyces_mrakii,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Phaeosphaeriaceae;g__Leptospora;s__unidentified,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Russulales;f__Bondarzewiaceae;g__Heterobasidion;s__Heterobasidion_parviporum,k__Fungi;p__Ascomycota;c__unidentified;o__unidentified;f__unidentified;g__unidentified;s__unidentified.1,...,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Xylariales;f__Xylariaceae;g__Barrmaelia;s__Barrmaelia_rappazii,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Tricholomataceae;g__Macrocystidia;s__Macrocystidia_cucumis,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Tricholomataceae;g__Macrocystidia;s__Macrocystidia_cucumis.1,k__Fungi;p__Ascomycota;c__Pezizomycetes;o__Pezizales;f__Pezizaceae;g__Iodophanus;s__Iodophanus_testaceus,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__unidentified;g__unidentified;s__unidentified.28,k__Fungi;p__Ascomycota;c__Pezizomycetes;o__Pezizales;f__Pyronemataceae;g__Wilcoxina;s__unidentified,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__unidentified.3,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Mycosphaerellaceae;g__Passalora;s__Passalora_passaloroides,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Agaricales;f__Pterulaceae;g__Pterula;s__Pterula_gracilis,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Auriculariales;f__unidentified;g__unidentified;s__unidentified.1
SAMPLE_NAME,,,,,,,,,,,,,,,,,,,,,
Lavaux_2021-09-30_must_Plot4_Row9/Vine20_ID6,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot15_Row3/Vine20_ID121b,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot17_Row13/Vine20_ID171,0.0,0.0,0.0,0.0,3.0,227.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot5_Row6/Vine30_ID37b,0.0,0.0,0.0,209.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot11_Row6/Vine22_ID66,0.0,0.0,0.0,64.0,0.0,106.0,0.0,3.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lavaux_2021-09-30_must_Plot18_Row2/Vine7_ID185,0.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot20_Row7/Vine20_ID207,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot9_Row11/Vine22_ID54,0.0,0.0,0.0,103.0,0.0,12.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Some metadata

In [7]:
md_soil = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_Soil.tsv', sep='\t')
md_soil = md_soil[['SAMPLE_NAME', 'Gravel_Percentage',
       'Gravel_Estimate', 'Organic_Matter', 'C_total', 'Clay_Percentage',
       'Silt_Percentage', 'Sand_Percentage', 'N_total', 'CN_ratio', 'pH']]
md_soil.head()

,SAMPLE_NAME,Gravel_Percentage,Gravel_Estimate,Organic_Matter,C_total,Clay_Percentage,Silt_Percentage,Sand_Percentage,N_total,CN_ratio,pH
0,Lavaux_2021-08-31_soil_Plot4,>30%,high,1.9,3.2775,32.9,41.1,26.0,0.13,8.3,7.55
1,Lavaux_2021-08-31_soil_Plot5,>30%,high,1.6,2.7600,29.5,38.5,32.1,0.11,8.7,7.53
2,Lavaux_2021-08-31_soil_Plot9,>30%,high,1.7,2.9325,22.9,42.4,34.7,0.11,9.5,7.55
3,Lavaux_2021-08-31_soil_Plot11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.47
4,Lavaux_2021-08-31_soil_Plot12,>30%,high,3.1,5.3475,34.0,41.3,24.7,0.18,10.0,7.40


In [8]:
md_management = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_Management.tsv', sep='\t')
md_management = md_management[['SAMPLE_NAME', 'Soil_thickness', 'Soil_type', 'Soil_depth',
       'Hydromorphie', 'Hydromorphie_code', 'Geology', 
       'Phenology_BudBreak', 'Phenology_Flowering',
       'Hail_Damage', 'Herbicide', 'Mildew', 'Grape_Yield_Estimate',
       'Pruning_Weight']]
md_management.head()

,SAMPLE_NAME,Soil_thickness,Soil_type,Soil_depth,Hydromorphie,Hydromorphie_code,Geology,Phenology_BudBreak,Phenology_Flowering,Hail_Damage,Herbicide,Mildew,Grape_Yield_Estimate,Pruning_Weight
0,Lavaux_2021-08-31_bark_Plot4,calcosol,Calcaire,100-180,Slightly_redox,1,Molasse,105,170,no,band,yes,medium,49.71
1,Lavaux_2021-08-31_bark_Plot5,calcosol,Calcaire,100-180,no_redox,0,Molasse,104,167,no,band,no,medium,57.41
2,Lavaux_2021-08-31_bark_Plot9,calcosol,Calcaire,100-180,no_redox,0,Molasse,111,172,no,almost_total,yes,low,38.99
3,Lavaux_2021-08-31_bark_Plot11,calcosol,Calcaire,100-180,Slightly_redox,1,Gravel_moraine,109,173,yes,band,yes,low,47.67
4,Lavaux_2021-08-31_bark_Plot12,colluviosol,Calcique,above_150,Slightly_redox,1,Colluvium,107,173,no,almost_total,yes,medium,54.09


In [9]:
md_df = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv', sep='\t')
md_df = md_df[md_df['sample_type']=='must']

# Plot metadata
plot_df = md_df[['SAMPLE_NAME', 'Altitude', 'Average_slope', 'Exposition','Average_radiation']].set_index('SAMPLE_NAME')
categorical_cols = ['Exposition']
plot_df_dummies = pd.get_dummies(plot_df, columns=categorical_cols, drop_first=True)
# Convert boolean columns to integers (0 or 1)
boolean_columns = plot_df_dummies.select_dtypes(include='bool').columns
plot_df_dummies[boolean_columns] = plot_df_dummies[boolean_columns].astype(int)

plot_df_dummies.tail()

,Altitude,Average_slope,Average_radiation,Exposition_South-West
SAMPLE_NAME,,,,
Lavaux_2023-09-20_must_Plot17_RepB,390,20,912.5,1
Lavaux_2023-09-20_must_Plot17_RepC,390,20,912.5,1
Lavaux_2023-09-20_must_Plot17_RepD,390,20,912.5,1
Lavaux_2023-09-20_must_Plot17_RepE,390,20,912.5,1
Lavaux_2021-09-30_must_Plot5_Row6/Vine30_ID37,400,30,912.5,1


In [10]:
md_df = pd.read_csv('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_Climate.tsv', sep='\t')
md_df_soil = md_df[md_df['sample_type']=='must']
# Climate metadata
climate_df = md_df_soil[['SAMPLE_NAME', 'median_temperature', 'cv_temperature', 'median_rh', 'cv_rh' ]].set_index('SAMPLE_NAME')
climate_df.head()

,median_temperature,cv_temperature,median_rh,cv_rh
SAMPLE_NAME,,,,
Lavaux_2021-08-31_must_Plot4,18.650958,22.587667,72.792393,16.567322
Lavaux_2021-08-31_must_Plot5,19.047366,22.104867,70.192394,16.331714
Lavaux_2021-08-31_must_Plot9,18.256629,22.831881,73.077585,16.570451
Lavaux_2021-08-31_must_Plot11,18.541438,23.246939,72.501870,16.118351
Lavaux_2021-08-31_must_Plot12,18.244984,23.615955,72.934711,16.661423


# Plant Pathogens

In [11]:
# UNITE fungal list
unite_df = pd.read_csv('/home/lfloerl/microterroir/Microbiome/ITS/UNITE-FungalTraits.csv')

# Step 1: Filter unite_df for rows where primary_lifestyle is 'plant_pathogen'
filtered_unite_df = unite_df[unite_df['primary_lifestyle'].str.contains('plant_pathogen', na=False)]
# Step 2: Extract GENUS from the filtered unite_df
genus_list = filtered_unite_df['GENUS'].tolist()

len(genus_list)

13961

In [12]:
# Create a regex pattern to match any term in the list
pattern = "|".join(genus_list)
# Filter columns based on regex pattern matching the genus in the taxa names
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col)]
# Filter the dataframe to only include the columns that match the pattern
pathogen_df = df_ITS[columns_to_keep]
pathogen_df.shape

(317, 354)

In [13]:
pathogen_df.head()

,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Phaeosphaeriaceae;g__Leptospora;s__unidentified,k__Fungi;p__Basidiomycota;c__Agaricomycetes;o__Russulales;f__Bondarzewiaceae;g__Heterobasidion;s__Heterobasidion_parviporum,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Didymellaceae;g__Neoascochyta;s__Neoascochyta_exitialis,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Cucurbitariaceae;g__Pyrenochaeta;s__unidentified,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Didymellaceae;g__Didymella;s__Didymella_vitalbina,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Comoclathris;s__Comoclathris_rosae,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__unidentified;g__unidentified;s__unidentified,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Didymellaceae;g__Epicoccum;s__Epicoccum_nigrum,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__Alternaria_subcucurbitae,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__Alternaria_alternata,...,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Teichosporaceae;g__Teichospora;s__Teichospora_austroafricana,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Botryosphaeriales;f__unidentified;g__unidentified;s__unidentified.1,k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Dermateaceae;g__Naevala;s__Naevala_perexigua,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__unidentified;g__unidentified;s__unidentified.27,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Neocamarosporium;s__Neocamarosporium_halimiones,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Leptosphaeriaceae;g__Subplenodomus;s__Subplenodomus_iridicola,k__Fungi;p__Ascomycota;c__Sordariomycetes;o__Diaporthales;f__Valsaceae;g__unidentified;s__unidentified.3,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__unidentified;g__unidentified;s__unidentified.28,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Pleosporales;f__Pleosporaceae;g__Alternaria;s__unidentified.3,k__Fungi;p__Ascomycota;c__Dothideomycetes;o__Capnodiales;f__Mycosphaerellaceae;g__Passalora;s__Passalora_passaloroides
SAMPLE_NAME,,,,,,,,,,,,,,,,,,,,,
Lavaux_2021-09-30_must_Plot4_Row9/Vine20_ID6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot15_Row3/Vine20_ID121b,0.0,4.0,0.0,0.0,1.0,0.0,0.0,134.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot17_Row13/Vine20_ID171,0.0,6.0,0.0,0.0,0.0,0.0,0.0,148.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot5_Row6/Vine30_ID37b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lavaux_2021-09-30_must_Plot11_Row6/Vine22_ID66,3.0,6.0,0.0,0.0,0.0,0.0,2.0,460.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Fermentative Yeasts

In [14]:
terms = [
    "Saccharomyces", "Saccharomycodes", "Schizosaccharomyces",
    "Candida", "Torulaspora", "Debaryomyces", "Issatchenkia",
    "Pichia", "Kluyveromyces", "Metschnikowia", "Hanseniaspora",
    "Kloeckera", "Rhodotorula", "Brettanomyces", "Dekkera",
    "Zygosaccharomyces"]
# Create a regex pattern to match any term in the list
pattern = "|".join(terms)

# Filter columns based on regex pattern matching the genus in the taxa names
columns_to_keep = [col for col in df_ITS.columns if re.search(pattern, col)]
# Filter the dataframe to only include the columns that match the pattern
fermentative_df = df_ITS[columns_to_keep]

fermentative_df.shape

(317, 8)

In [18]:
!pwd

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/DifferentialAbundance/Berries


In [ ]:
fermentative_df.to_csv('FermentativeOrganisms_crOTU90_annotated.csv')